In [57]:
from herbie import Herbie
from datetime import datetime, timedelta
    
file_name = '/home/bran/SNOWPACK/input/Tioga/Tioga_test.smet'
data_index = 1 #index starts at 0
start_time_str = '2023-11-06 12:00:00'

for i in range(1):
    dt = datetime.fromisoformat(start_time_str)
    dt_query_str = dt.strftime('%Y-%m-%d %H:%M')
    output_timestamp = dt + timedelta(hours=i)
    output_timestamp_str = output_timestamp.strftime('%Y-%m-%dT%H:%M')
    hours_offset = i
    H = Herbie("2023-11-06 12:00",model="hrrr",fxx=hours_offset,save_dir="~/ESAC_herbiedata")
    ss = ":TMP:2 m|:RH:2 m|:WIND:10 m|:.GRD:10 m|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface"
    ss1 = ":TMP:2 m|:RH:2 m" # 2m readings
    ss2 = ":WIND:10 m|:.GRD:10 m" # 10m readings
    ss3 = ":DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface" # surf readings
    #print(H.inventory(searchString=":TMP:2 m|:RH:2 m|:WIND:10 m|:.GRD:10 m|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface"))
    # not sure about winds
    ds = H.xarray(ss1, remove_grib=False)
    ll = [(-119.02,37.64),(-119.25,37.91),(-118.73,37.45)]
    y=ds.herbie.nearest_points(points=ll)
    print('querying data for: ', dt_str, hours_offset, output_timestamp_str)
    y_d = y.to_dict()
    append_data(y_d, output_timestamp_str)
# pri#nt(ds)
#print(y)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Nov-06 12:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:260: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.parse_cf()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "t2m".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "r2".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.assign_y_x()


querying data for:  2023-11-06 12:00 0 2023-11-06T12:00
<class 'list'>
31
31


In [54]:
def append_data(y_d, output_timestamp_str):
    t2m = y_d['data_vars']['t2m']['data'][data_index]
    r2 = y_d['data_vars']['r2']['data'][data_index]
    t2m_formatted = round(t2m, 1) #2nd parameter is number of decimals
    r2_formatted = round(r2, 1)
    new_line_str = f"{output_timestamp_str} {t2m_formatted} {r2_formatted}\n"
    with open(file_name, 'r') as f:
        lines = f.readlines()
        print(type(lines))
        print(len(lines))
        new_lines = lines
        found= False
        for idx, line in enumerate(lines):
            if line[:16] == output_timestamp_str:
                new_lines[idx] = new_line_str
                found = True
                break
        if not found:
            new_lines += [new_line_str]
    print(len(new_lines))
    with open(file_name, 'w') as f:
        f.writelines(new_lines)

<_io.TextIOWrapper name='/home/bran/SNOWPACK/input/Tioga/Tioga_test.smet' mode='r' encoding='UTF-8'>